<a href="https://colab.research.google.com/github/PreethiGorantla/IMBD-sentiment-Analysis/blob/main/SentimentAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q datasets tensorflow keras
from datasets import load_dataset

# Load IMDb dataset
imdb = load_dataset("imdb")

# Check one example
print(imdb["train"][0])


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Extract texts and labels
train_texts = imdb["train"]["text"]
train_labels = np.array(imdb["train"]["label"])
test_texts  = imdb["test"]["text"]
test_labels = np.array(imdb["test"]["label"])

# Tokenize
num_words = 20000   # keep top 20k words
max_len = 200       # pad/truncate reviews to 200 words

tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
tokenizer.fit_on_texts(train_texts)

X_train = tokenizer.texts_to_sequences(train_texts)
X_test  = tokenizer.texts_to_sequences(test_texts)

X_train = pad_sequences(X_train, maxlen=max_len, padding="post", truncating="post")
X_test  = pad_sequences(X_test, maxlen=max_len, padding="post", truncating="post")

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)


Train shape: (25000, 200)
Test shape: (25000, 200)


In [ ]:
# Download GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip

# Load GloVe embeddings into a dictionary
embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = coeffs

print("Words loaded:", len(embedding_index))


--2025-08-25 15:44:31--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-08-25 15:44:31--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-08-25 15:44:32--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
embedding_dim = 100  # because we are using glove.6B.100d.txt
word_index = tokenizer.word_index
vocab_size = min(num_words, len(word_index)) + 1  # +1 for padding token

# Initialize embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))

for word, i in word_index.items():
    if i >= vocab_size:
        continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        embedding_matrix[i] = np.random.normal(scale=0.6, size=(embedding_dim,))

print("Embedding matrix shape:", embedding_matrix.shape)


Embedding matrix shape: (20001, 100)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=False))  # Freeze GloVe embeddings

model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))  # Binary classification

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │     2,000,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,000,100 (7.63 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 2,000,100 (7.63 MB)

In [ ]:
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    input_length=max_len,
                    trainable=True))  # <-- currently frozen


In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=vocab_size,
                    output_dim=embedding_dim,
                    weights=[embedding_matrix],
                    trainable=True))  # fine-tune embeddings

model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))  # binary output, shape=(None,1)

model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)         │ ?                      │     2,000,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,000,100 (7.63 MB)

 Trainable params: 2,000,100 (7.63 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
print(train_labels.shape)  # should be (25000,)
print(X_train.shape)       # should be (25000, 200)


(25000,)
(25000, 200)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train, train_labels,
    validation_split=0.2,
    epochs=10,
    batch_size=128,
    callbacks=[early_stop]
)


Epoch 1/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 260s 2s/step - accuracy: 0.6214 - loss: 0.6624 - val_accuracy: 0.1110 - val_loss: 0.9922
Epoch 2/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 228s 1s/step - accuracy: 0.6311 - loss: 0.6510 - val_accuracy: 0.1602 - val_loss: 0.9795
Epoch 3/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 231s 1s/step - accuracy: 0.6695 - loss: 0.6080 - val_accuracy: 0.7012 - val_loss: 0.7270
Epoch 4/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 258s 1s/step - accuracy: 0.7918 - loss: 0.4783 - val_accuracy: 0.2042 - val_loss: 0.9785
Epoch 5/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 254s 1s/step - accuracy: 0.8478 - loss: 0.3697 - val_accuracy: 0.7178 - val_loss: 0.6843
Epoch 6/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 264s 1s/step - accuracy: 0.9011 - loss: 0.2563 - val_accuracy: 0.7626 - val_loss: 0.6692
Epoch 7/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 269s 1s/step - accuracy: 0.9302 - loss: 0.1940 - val_accuracy: 0.7118 - val_loss: 0.8014
Epoch 8/10
157/157 ━━━━━━━━━━━━━━━━━━━━ 252s 1s/step - accuracy: 0.9490 - loss: 0.1474 - val_accu

In [ ]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test, test_labels, batch_size=128)
print(f"Test Accuracy: {test_acc*100:.2f}%")


196/196 ━━━━━━━━━━━━━━━━━━━━ 57s 293ms/step - accuracy: 0.8805 - loss: 0.2832
Test Accuracy: 84.04%


In [ ]:
from tensorflow.keras.layers import GRU, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense

model_gru = Sequential()
model_gru.add(Embedding(input_dim=vocab_size,
                        output_dim=embedding_dim,
                        weights=[embedding_matrix],
                        trainable=True))

# Use GRU instead of LSTM, optionally bidirectional
model_gru.add(Bidirectional(GRU(128, dropout=0.2, recurrent_dropout=0.2)))
model_gru.add(Dense(1, activation="sigmoid"))

model_gru.compile(optimizer="adam",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])

model_gru.summary()


NameError: name 'vocab_size' is not defined

In [ ]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history_gru = model_gru.fit(
    X_train, train_labels,
    validation_split=0.2,
    epochs=10,
    batch_size=128,
    callbacks=[early_stop]
)


NameError: name 'EarlyStopping' is not defined

In [ ]:
# Evaluate GRU model on test data
test_loss, test_acc = model_gru.evaluate(X_test, test_labels, batch_size=128)
print(f"GRU Test Accuracy: {test_acc*100:.2f}%")


196/196 ━━━━━━━━━━━━━━━━━━━━ 94s 479ms/step - accuracy: 0.8689 - loss: 0.3050
GRU Test Accuracy: 83.18%


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, ConfusionMatrixDisplay

# ✅ Get predicted probabilities
y_pred_prob = model_gru.predict(X_test, batch_size=128)

# ✅ Convert probabilities to class labels (binary classification: 0 or 1)
y_pred = (y_pred_prob > 0.5).astype("int32")

# -------------------------------
# 1. Confusion Matrix
# -------------------------------
cm = confusion_matrix(test_labels, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0,1])
disp.plot(cmap=plt.cm.Blues)
plt.title("Confusion Matrix")
plt.show()

# Classification report (Precision, Recall, F1)
print(classification_report(test_labels, y_pred))

# -------------------------------
# 2. ROC Curve
# -------------------------------
fpr, tpr, thresholds = roc_curve(test_labels, y_pred_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(6,6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc="lower right")
plt.show()


NameError: name 'model_gru' is not defined

In [ ]:
def predict_sentiment_gru(text):
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_len, padding="post", truncating="post")
    pred = model_gru.predict(padded)[0][0]
    return "Positive" if pred > 0.5 else "Negative"

# Example
print(predict_sentiment_gru("This movie was absolutely fantastic! I loved it."))
print(predict_sentiment_gru("Terrible movie. Waste of time and money."))


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
Negative


In [ ]:
# New reviews
reviews = [
    "I really enjoyed this film, the story was gripping and the actors were amazing.",
    "The plot was so boring and predictable, I almost fell asleep.",
    "Absolutely stunning visuals and a heartwarming story. Highly recommend!",
    "Terrible movie. Poor acting, bad script, and way too long.",
    "It was okay, some parts were good but overall not very impressive."
]

# Predict sentiments
for review in reviews:
    sentiment = predict_sentiment_gru(review)
    print(f"Review: {review}\nPredicted Sentiment: {sentiment}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
Review: I really enjoyed this film, the story was gripping and the actors were amazing.
Predicted Sentiment: Positive

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Review: The plot was so boring and predictable, I almost fell asleep.
Predicted Sentiment: Negative

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Review: Absolutely stunning visuals and a heartwarming story. Highly recommend!
Predicted Sentiment: Positive

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
Review: Terrible movie. Poor acting, bad script, and way too long.
Predicted Sentiment: Negative

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
Review: It was okay, some parts were good but overall not very impressive.
Predicted Sentiment: Positive

